In [ ]:
import os
import random
from datetime import datetime
import ast

import gc
import yaml
from tqdm import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch.utils.tensorboard import SummaryWriter

from preprocessing.final_preprocessing import serve_data

from models.transformer_encoder import TransEncoder
from models.baseline import BaseLineModel

from ddpm.ddpm import GaussianDiffusion1D

from evaluation.pca_tsne import visualize_pca_tsne
from evaluation.jsd import compute_jsd
from evaluation.rmse import rmse

gc.collect()

random.seed(42)
np.random.seed(42)

In [ ]:
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# Parameters

In [ ]:
config_path = "./config.yml"
with open(config_path, 'r') as f:
    config = yaml.full_load(f)

date = datetime.today().strftime('%Y-%m-%d-%H-%M')
device = 'cuda' if torch.cuda.is_available() else 'cpu'

seq_len = config["seq_len"]
epochs = config["epochs"]
timesteps = config["timesteps"]
batch_size_config = config["batch_size"]
latent_dim = config["latent_dim"]
num_layers = config["num_layers"]
n_heads = config["n_heads"]
beta_schedule = config["beta_schedule"]
objective = config["objective"]
model_name = config["model_name"]
cond_model = config["cond_model"]
lr = float(config["lr"])
betas = tuple_of_floats = ast.literal_eval(config["betas"])
save_rate = int(config["save_rate"])

img_rows = int(config["img_rows"])
img_cols = int(config["img_cols"])
channels = int(config["channels"])
n_feat = int(config["n_feat"])
num_customers = int(config["num_customers"])

assert model_name in ["BaseLine", "TransEncoder"], "Chosen  model was not valid, the options are BaseLine or TransEncoder"
assert cond_model in {"mlp", "te", "fft", "stft"}, "Chosen conditioning model was not valid, the options are mlp, te, fft and stft"

log_file_name = f"{model_name}_{cond_model}_{str(date)}"
tb_writer = f"./logging/tensorboard/{log_file_name}"

In [ ]:
cond_model

# Data

In [ ]:
train_loader, test_loader, train_cols, test_cols, test_data, features, cond_features = serve_data(seq_len=seq_len, batch_size=batch_size_config)

# Model

In [ ]:
if model_name == "TransEncoder":
    model = model = TransEncoder(features = features, latent_dim = latent_dim,num_heads = n_heads, num_layers = num_layers, cond_features=cond_features, cond_model=cond_model, device=device, seq_len=seq_len)
if model_name == "BaseLine":
    model = BaseLineModel(seq_len=seq_len, hidden_dim=latent_dim, cond_dim=cond_features, cond_model=cond_model, device=device, channels=features)
    

ddpm = GaussianDiffusion1D(model, seq_length = seq_len, timesteps = timesteps, objective = objective, loss_type = 'l2', beta_schedule = beta_schedule)
ddpm = ddpm.to(device)

optim = torch.optim.Adam(ddpm.parameters(), lr = lr, betas = betas)
writer = SummaryWriter(tb_writer)

## Train

In [ ]:
run_path = f"./logging/weights/{log_file_name}"
os.makedirs(run_path)

rmse_list = []
lowest_loss = 100000
step=0

for running_epoch in tqdm(range(epochs)):
    running_loss = 0.0
    for i, (data, cond_data) in enumerate(train_loader):
        data = data.to(device)
        
        cond_data = cond_data.float()
        cond_data = cond_data.to(device)
                
        batch_size = data.shape[0]
        
        optim.zero_grad()
        
        loss = ddpm(data, cond_data)
        loss.backward()
        
        optim.step()
        
        running_loss += loss.item()

        
        if loss.item() < lowest_loss:
            lowest_loss = loss.item()
            best_model_params = model.state_dict()
        
        writer.add_scalar("Batch Training Loss", loss, global_step=step)
        step += 1
    
    avg_epoch_loss = running_loss / len(train_loader)
    writer.add_scalar("Epoch Training Loss", avg_epoch_loss, running_epoch)
    
    if running_epoch % 5 ==0:        
        with torch.no_grad():
            sample = ddpm.sample(batch_size_config)
        
        p = sample.cpu()
        q = torch.from_numpy(test_data[:batch_size_config])
            
        rmse_res = rmse(p,q).item()
        rmse_list.append(rmse_res)
        
        writer.add_scalar('Epoch Training RMSE', rmse_res, running_epoch)
        
        print(f'Epoch: {running_epoch+1}, Epoch Loss: {avg_epoch_loss:.4f}, RMSE: {rmse_res:.4f}')
    
    else:
        print(f'Epoch: {running_epoch+1}, Epoch Loss: {avg_epoch_loss:.4f}')
            
            
    if running_epoch % save_rate == 0:
        torch.save({
            'epoch': running_epoch+1,
            'diffusion_state_dict': ddpm.state_dict(),
            'diffusion_optim_state_dict': optim.state_dict()
            }, os.path.join(f'{run_path}', f'ep_{running_epoch}_mse_{avg_epoch_loss:.3f}_weights.pth'))

torch.save(best_model_params, os.path.join(f'{run_path}', 'best_model_weights.pth'))

## Sampling

In [ ]:
with torch.no_grad():
    samples = ddpm.sample(batch_size_config)
    samples = samples.cpu().numpy()
    samples = samples.transpose(0, 2, 1)

print(f"Samples shape: {samples.shape}")

# Evaluation
## Without Conditioning
### Train

In [ ]:
real_data_train, real_cond_data_train = next(iter(train_loader))
real_data_test, real_cond_data_test = next(iter(test_loader))

real_cond_data_train.shape, real_data_test.shape

In [ ]:
train_customer_ids = random.sample(train_cols, num_customers)

In [ ]:
for idx, feature in enumerate(train_customer_ids):
    real_feature = np.expand_dims(real_data_train[:, idx, :].cpu().numpy(), axis=-1)
    gen_feature = np.expand_dims(samples[:, :, idx], axis=-1)
    
    visualize_pca_tsne(ori_data=real_feature, fake_data=gen_feature, seq_len=seq_len, train_test="train", filename=log_file_name, cond=False, feature=feature)

### Test

In [ ]:
test_customer_ids = random.sample(test_cols, num_customers)

In [ ]:
for idx, feature in enumerate(test_customer_ids):
    real_feature = np.expand_dims(real_data_test[:, idx, :].cpu().numpy(), axis=-1)
    gen_feature = np.expand_dims(samples[:, :, idx], axis=-1)
    
    visualize_pca_tsne(ori_data=real_feature, fake_data=gen_feature, seq_len=seq_len, train_test="test", filename=log_file_name, cond=False, feature=feature)

### Jensen-Shannon Divergence

In [ ]:
customer_ids = train_customer_ids + test_customer_ids

In [ ]:
train_jsd_no_con_list = []
test_jsd_no_con_list = []

for idx, feature in enumerate(customer_ids):
    real_feature_train = real_data_train[:, idx, :].cpu().numpy()
    real_feature_test = real_data_test[:, idx, :].cpu().numpy()
    gen_feature = samples[:, :, idx]
    
    jsd_no_con_train = compute_jsd(real_feature_train, samples)
    jsd_no_con_test = compute_jsd(real_feature_test, samples)
    
    train_jsd_no_con_list.append(jsd_no_con_train)
    test_jsd_no_con_list.append(jsd_no_con_test)

## With Conditioning

In [ ]:
real_cond_data_train.shape

In [ ]:
with torch.no_grad():
    cond_samples_train = ddpm.sample(real_cond_data_train.shape[0], real_cond_data_train.to(device))
    cond_samples_train = cond_samples_train.cpu().numpy()
    cond_samples_train = cond_samples_train.transpose(0, 2, 1)

print(f"Samples shape: {cond_samples_train.shape}")

### T-SNE and PCA

In [ ]:
for idx, feature in enumerate(train_customer_ids):
    real_feature = np.expand_dims(real_data_train[:, idx, :].cpu().numpy(), axis=-1)
    gen_feature = np.expand_dims(cond_samples_train[:, :, idx], axis=-1)
    
    visualize_pca_tsne(ori_data=real_feature, fake_data=gen_feature, seq_len=seq_len, train_test="train", filename=log_file_name, cond=True, feature=feature)

In [ ]:
with torch.no_grad():
    cond_samples_test = ddpm.sample(real_cond_data_test.shape[0], real_cond_data_test.to(device))
    cond_samples_test = cond_samples_test.cpu().numpy()
    cond_samples_test = cond_samples_test.transpose(0, 2, 1)

print(f"Samples shape: {cond_samples_test.shape}")

In [ ]:
for idx, feature in enumerate(test_customer_ids):
    real_feature = np.expand_dims(real_data_train[:, idx, :].cpu().numpy(), axis=-1)
    gen_feature = np.expand_dims(cond_samples_test[:, :, idx], axis=-1)
    
    visualize_pca_tsne(ori_data=real_feature, fake_data=gen_feature, seq_len=seq_len, train_test="test", filename=log_file_name, cond=True, feature=feature)

### Jensen-Shannon Divergence

In [ ]:
train_jsd_con_list = []
test_jsd_con_list = []

for idx, feature in enumerate(customer_ids):
    real_feature_train = real_data_train[:, idx, :].cpu().numpy()
    real_feature_test = real_data_test[:, idx, :].cpu().numpy()
    gen_feature_train = cond_samples_train[:, :, idx]
    gen_feature_test = cond_samples_test[:, :, idx]
    
    jsd_con_train = compute_jsd(real_feature_train, gen_feature_train)
    jsd_con_test = compute_jsd(real_feature_test, gen_feature_test)
    
    train_jsd_con_list.append(jsd_con_train)
    test_jsd_con_list.append(jsd_con_test)

## Viz

In [ ]:
data = {
    "train_jsd_con": train_jsd_con_list,
    "train_jsd_no_con": train_jsd_no_con_list,
    "test_jsd_con": test_jsd_con_list,
    "test_jsd_no_con": test_jsd_no_con_list
}

jsd_data = pd.DataFrame(data, index=customer_ids)
jsd_data.head()


In [ ]:
if not os.path.isdir('./logging/plots/JSD/'):
    os.makedirs('./logging/plots/JSD/')

jsd_data.plot(kind='bar', figsize=(10, 6))

plot_dir = f'./logging/plots/JSD/'
plt.savefig(os.path.join(plot_dir, log_file_name+".png", ))
plt.show()

# Logging

In [ ]:
log_dir = "./logging/logs"
log_file_path = os.path.join(log_dir, log_file_name + ".txt")

jsd_data.to_csv(os.path.join(log_dir, log_file_name + ".csv"))

with open(log_file_path, 'w') as log_file:
    log_file.write("Config:\n")
    for key, value in config.items():
        log_file.write(f"{key}: {value}\n")
    
    log_file.write("\n\nRMSE Values:\n")
    for rmse in rmse_list:
        log_file.write(f"{rmse}\n")
    
    log_file.write("\n\nJSD Values:\n")
    log_file.write(str(os.path.join(log_dir, log_file_name + ".csv")))

print(f"Log file created at: {log_file_path}")

In [ ]:
#best_model = torch.load("./logging/weights/TransEncoder_te_2025-02-19-19-39/best_model_weights.pth")